In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import logging

import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd
from scipy.signal import convolve2d, detrend

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("figure", figsize=(12,10))
plt.rc("font", size=14)


# Import modules
import numpy as np
import numpy.ma as ma
import os, sys, time, re
from datetime import date
import scipy
import calendar
import math
import netCDF4 as nc
from scipy import signal,fftpack, interpolate
from scipy.interpolate import griddata

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, silence_logs='error')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:8787/status,
Dashboard: http://137.129.155.67:8787/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: inproc://137.129.155.67/7522/1,Workers: 1
Dashboard: http://137.129.155.67:8787/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: inproc://137.129.155.67/7522/4,Total threads: 4
Dashboard: http://137.129.155.67:43837/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
###############################################################################
# realtime_filter.py
#
# Written by Jared Rennie, NCICS / CICS-NC
# E-mail: jared@cicsnc.org
#
# **DESCRIPTION**
# Performs Fourier filtering of a variable in time. Uses observational data
# as well as forecast data from CFS. Forward filtering is performed on the 
# data, certain parts are set to zero (based on metrics of the filters), and
# then inverse filtering is performed, which provides the final output.
#
# This is PHASE ONE of the Madden-Julian Oscillation code suite.
###############################################################################


#################################################
# MODULE: kf_filter
#################################################
def kf_filter(inData,obsPerDay,tMin,tMax,kMin,kMax,hMin,hMax,waveName):
	timeDim = inData.shape[0]
	lonDim = inData.shape[1]
	
	# Reshape data from [time,lon] to [lon,time]
	originalData=np.zeros([lonDim,timeDim],dtype='f')
	for counterX in range(timeDim):
	    test=0
	    for counterY in range(lonDim-1,-1,-1):
	        originalData[test,counterX]=inData[counterX,counterY]
	        test+=1
	
	# Detrend the Data
	detrendData=np.zeros([lonDim,timeDim],dtype='f')
	for counterX in range(lonDim):
	    detrendData[counterX,:]=signal.detrend(originalData[counterX,:])
	
# 	# Taper 
# 	taper=signal.tukey(timeDim,0.05,True)
# 	taperData=np.zeros([lonDim,timeDim],dtype='f')
# 	for counterX in range(lonDim):
# 	    taperData[counterX,:]=detrendData[counterX,:]*taper
	
	# Perform 2-D Fourier Transform
	fftData=np.fft.rfft2(detrendData)
	kDim=lonDim 
	freqDim=round(fftData.shape[1])
	
	# Find the indeces for the period cut-offs
	jMin = int(round( ( timeDim * 1. / ( tMax * obsPerDay ) ), 0 ))
	jMax = int(round( ( timeDim * 1. / ( tMin * obsPerDay ) ), 0 ))
	jMax = min( ( jMax, freqDim ) )

	# Find the indices for the wavenumber cut-offs
	# This is more complicated because east and west are separate
	if( kMin < 0 ):
	    iMin = round( ( kDim + kMin ), 3 )
	    iMin = max( ( iMin, ( kDim / 2 ) ) )
	else:
	    iMin = round( kMin, 3 )
	    iMin = min( ( iMin, ( kDim / 2 ) ) )

	if( kMax < 0 ):
	    iMax = round( ( kDim + kMax ), 3 )
	    iMax = max( ( iMax, ( kDim / 2 ) ) )
	else:
	    iMax = round( kMax, 3 )
	    iMax = min( ( iMax, ( kDim / 2 ) ) )
	  
	# set the appropriate coefficients to zero
	iMin=int(iMin)
	iMax=int(iMax)
	jMin=int(jMin)
	jMax=int(jMax)
	if( jMin > 0 ):
	    fftData[:, :jMin-1 ] = 0
	if( jMax < ( freqDim - 1 ) ):
	    fftData[:, jMax+1: ] = 0

	if( iMin < iMax ):
	    # Set things outside the range to zero, this is more normal
	    if( iMin > 0 ):
	        fftData[:iMin-1, : ] = 0
	    if( iMax < ( kDim - 1 ) ):
	        fftData[iMax+1:, : ] = 0
	else:
	    # Set things inside the range to zero, this should be somewhat unusual
	    fftData[iMax+1:iMin-1, : ] = 0
	
	# Find constants
	PI = math.pi
	beta = 2.28e-11
	if hMin != -999:
	    cMin = float( 9.8 * float(hMin) )**0.5
	else:
	    cMin=hMin
	if hMax != -999:
	    cMax = float( 9.8 * float(hMax) )**0.5
	else:
	    cMax=hMax
	c = np.array([cMin,cMax])
	spc = 24 * 3600. / ( 2 * PI * obsPerDay ) # seconds per cycle
	
	# Now set things to zero that are outside the Kelvin dispersion
	for i in range(0,kDim):
		# Find Non-Dimensional WaveNumber (k)
		if( i > ( kDim / 2 ) ):
			# k is negative
			k = ( i - kDim  ) * 1. / (6.37e6) # adjusting for circumfrence of earth
		else:
			# k is positive
			k = i * 1. / (6.37e6) # adjusting for circumfrence of earth
		
		# Find Frequency
		freq = np.array([ 0, freqDim * (1. / spc) ]) #waveName='None'
		jMinWave = 0
		jMaxWave = freqDim	
		if waveName.lower() == "kelvin":
		    freq = k * c
		if waveName.lower() == "er":
		    freq = -beta * k / ( k**2 + 3. * beta / c )
		if waveName.lower() == "ig1":
		    freq = ( 3 * beta * c + k**2 * c**2 )**0.5
		if waveName.lower() == "ig2":
		    freq = ( 5 * beta * c + k**2 * c**2 )**0.5
		if waveName.lower() == "mrg" or waveName.lower()=="ig0":   	
			if( k == 0 ):
				freq = ( beta * c )**0.5
			else:
				if( k > 0):
					freq = k * c * ( 0.5 + 0.5 * ( 1 + 4 * beta / ( k**2 * c ) )**0.5 )
				else:
					freq = k * c * ( 0.5 - 0.5 * ( 1 + 4 * beta / ( k**2 * c ) )**0.5 )	
		
		# Get Min/Max Wave 
		if(hMin==mis):
		    jMinWave = 0
		else:
		    jMinWave = int( math.floor( freq[0] * spc * timeDim ) )

		if(hMax==mis):
		    jMaxWave = freqDim
		else:
		    jMaxWave = int( math.ceil( freq[1] * spc * timeDim ) )

		jMaxWave = max(jMaxWave, 0)
		jMinWave = min(jMinWave, freqDim)
		
		# set the appropriate coefficients to zero
		i=int(i)
		jMinWave=int(jMinWave)
		jMaxWave=int(jMaxWave)
		if( jMinWave > 0 ):
		    fftData[i, :jMinWave-1] = 0
		if( jMaxWave < ( freqDim - 1 ) ):
		    fftData[i, jMaxWave+1:] = 0
	
	# perform the inverse transform to reconstruct the data
	returnedData=np.fft.irfft2(fftData) 
	
	# Reshape data from [lon,time] to [time,lon]
	outData=np.zeros([timeDim,lonDim],dtype='f')
	for counterX in range(returnedData.shape[1]):
	    test=0
	    for counterY in range(lonDim-1,-1,-1):
	        outData[counterX,counterY]=returnedData[test,counterX] 
	        test+=1
    
    # Return Result
	return outData

In [3]:
###################################################################################
def createArray(year) :
    _ds_m1 = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year-1)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds_m1 = _ds_m1.isel(time = slice(-addDay*spd,None))
    _ds = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds1 = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year+1)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds1 = _ds1.isel(time = slice(None,addDay*spd))

    ds = xr.concat([_ds_m1,_ds,_ds1], dim='time', coords='minimal', compat='override')
    
    return ds

In [4]:
indir_data = '/cnrm/tropics/user/peyrille/Stage_PFE_Erwan/data/OLR/'
filenames = np.arange(2009,2010)
var = 'OLR'
units = "W/m^2"
var_file = 'olr_anom_wkfilter_'
mis = -999

addDay = 360
spd = 1

for f in filenames:
    date_Re = pd.date_range(start = (str(f) + '-01-01'), end = (str(f + 1) + '-01-01'), freq = '24H', closed = 'left')
    ds = createArray(f)
    ds = ds.sel(lat = slice(-15,15))
    infile = ds.load()
infile

<xarray.Dataset>
Dimensions:   (time: 1085, lat: 13, lon: 144)
Coordinates:
  * lat       (lat) float32 -15.0 -12.5 -10.0 -7.5 -5.0 ... 7.5 10.0 12.5 15.0
  * lon       (lon) float32 -177.5 -175.0 -172.5 -170.0 ... 175.0 177.5 180.0
  * time      (time) datetime64[ns] 2008-01-07 2008-01-08 ... 2010-12-26
Data variables: (12/13)
    aew2_10   (time, lat, lon) float32 -3.709 -5.114 -9.516 ... 3.072 7.328
    er_ki     (time, lat, lon) float32 -3.794 -6.618 -9.104 ... -4.537 -1.355
    er_wk     (time, lat, lon) float32 -2.199 -3.125 -4.173 ... -4.34 -2.511
    klv_sk    (time, lat, lon) float32 -27.94 -21.08 -12.56 ... -11.28 -9.525
    klv_wk    (time, lat, lon) float32 -23.85 -16.9 -8.308 ... -9.363 -7.425
    low       (time, lat, lon) float32 -11.38 -11.08 -10.73 ... -2.547 -5.512
    ...        ...
    mjo_wk    (time, lat, lon) float32 -21.06 -18.79 -16.5 ... -0.7595 -2.335
    mrg       (time, lat, lon) float32 -13.65 -15.4 -15.45 ... 1.75 4.046
    mrg_td    (time, lat, lon) float32 0.3151 3.123 1.544 ... 11.15 15.39 22.02
    olr       (time, lat, lon) float32 179.8 171.5 145.2 ... 283.0 284.2 284.5
    olr_anom  (time, lat, lon) float32 -39.92 -47.52 -74.62 ... 9.028 9.862
    td        (time, lat, lon) float32 -5.07 -2.974 -3.377 ... 0.9168 0.3516
Attributes:
    history:  Fri Jun 18 10:15:55 2021: ncks -O --mk_rec_dmn time olr_anom_wk...
    NCO:      netCDF Operators version 4.9.2 (Homepage = http://nco.sf.net, C...

In [5]:
#################################################
# BEGIN PROGRAM
#################################################
start=time.time()

# Set Pre-Defined Arguments
mis = -999
obsPerDay = 1
algoName='cfs'
minLat=-45
maxLat=45

unit='W m-2'

#################################################
# Read in Observational Data
#################################################

lat = infile.variables['lat'][:]
lon = infile.variables['lon'][:]
obslon2d, obslat2d = np.meshgrid(lon, lat)

# only need values between -45 and 45 latitude
valid_lats=np.where(np.logical_and(lat>=minLat,lat<=maxLat))[0]
lat=lat[valid_lats]

obsTime = infile.variables['time'][:]

obsVar = 'olr_anom'
obsData = infile.variables[obsVar][:,valid_lats,:]
obsData

<xarray.Variable (time: 1085, lat: 13, lon: 144)>
array([[[ -39.915924  ,  -47.52356   ,  -74.615845  , ...,
         -109.666885  ,  -90.94217   ,  -43.10222   ],
        [ -30.533112  ,  -15.483261  ,  -44.350525  , ...,
          -82.11908   ,  -76.89229   ,  -65.690384  ],
        [  25.50447   ,   -1.5575562 ,   20.813614  , ...,
          -41.563858  ,  -28.25624   ,   -2.291504  ],
        ...,
        [   3.5997314 ,  -13.22699   ,  -38.63922   , ...,
          -18.014832  ,   -2.8446655 ,   -1.7725525 ],
        [   9.735992  ,    9.736267  ,    8.3793335 , ...,
            2.236206  ,    3.6584473 ,    5.0078125 ],
        [   5.3338623 ,    6.0181274 ,    6.1020203 , ...,
            0.2954712 ,    3.860443  ,    5.6737976 ]],

       [[ -26.345627  ,  -74.375     ,  -43.48024   , ...,
           40.103897  ,  -55.75377   ,  -91.46489   ],
        [ -51.065002  ,  -80.807556  ,  -66.21724   , ...,
          -32.601334  ,  -75.68692   ,  -71.430176  ],
        [ -56.516647  ,  -54.3387    ,  -18.944443  , ...,
          -35.275726  ,  -29.489105  ,  -29.990814  ],
...
        [  15.431091  ,    8.686188  ,   -3.176468  , ...,
           12.386932  ,   15.25499   ,   18.754608  ],
        [   2.2426147 ,   -2.605835  ,    7.3239136 , ...,
            2.7599792 ,  -22.122086  ,  -29.280731  ],
        [ -16.987915  ,  -10.440094  ,    8.41626   , ...,
            3.407898  ,    3.5073853 ,  -19.423584  ]],

       [[ -85.72604   ,  -36.28618   ,   -7.283844  , ...,
          -27.307556  ,  -34.22232   ,  -48.90123   ],
        [ -58.63704   ,  -38.056946  ,  -23.201187  , ...,
          -18.111511  ,  -13.506134  ,  -63.664642  ],
        [   9.625595  ,   36.82422   ,   42.103287  , ...,
           16.495407  ,    5.351227  ,    0.44073486],
        ...,
        [  22.17218   ,   12.644241  ,    6.2104187 , ...,
           13.439941  ,   15.812958  ,   23.30989   ],
        [  13.79837   ,   10.002869  ,    4.27417   , ...,
           10.9496155 ,    7.9503784 ,   12.890442  ],
        [   9.568115  ,    5.8513794 ,   -2.9039612 , ...,
            7.711609  ,    9.028168  ,    9.862457  ]]], dtype=float32)
Attributes:
    valid_range:           [-32765  17235]
    parent_stat:           Individual Obs
    statistic:             Mean
    level_desc:            Other
    dataset:               NOAA Interpolated OLR
    precision:             2
    var_desc:              Outgoing Longwave Radiation
    units:                 W/m^2
    actual_range:          [ 64.75 344.5 ]
    unpacked_valid_range:  [  0. 500.]
    long_name:              olr anomaly

In [6]:
filtData = obsData.copy()
_filtData = filtData.copy()
#################################################
# Filter (ER)
#################################################
filtVar = 'olr'
print("\n##############################\nFiltering (ER)\n")
# filtData = infile
LF = filtData.copy()
LF_wavenumber=np.array([-10,0],dtype='f')
LF_period=np.array([120,999],dtype='f')
LF_depth=np.array([mis,mis],dtype='f')
LF_units=unit

er = filtData.copy()
# er_longname="Equatorial Rossby Waves in "+filtVar.upper()
# er_filter="Kiladis et al. (2009 Rev. Geophys.)"
er_wavenumber=np.array([-10,-1],dtype='f')
er_period=np.array([9.7,72],dtype='f')
er_depth=np.array([8,90],dtype='f')
er_units=unit

for lat_counter in range(0,len(lat)):
    print("###########################################################################################")
    print(filtData[:,lat_counter,:].max().values, '\n')
    print(lat_counter,lat[lat_counter])
    
    LF[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
    LF_period[0],LF_period[1],\
    LF_wavenumber[0],LF_wavenumber[1],\
    LF_depth[0],LF_depth[1],"LF")
    print('max : ', LF[:,lat_counter,:].max().values,'\n')
    
    print(filtData[:,lat_counter,:].max().values, '\n')
    _filtData[:,lat_counter,:] = filtData[:,lat_counter,:] - LF[:,lat_counter,:]
    print(_filtData[:,lat_counter,:].max().values, '\n')
    
    er[:,lat_counter,:]=kf_filter(_filtData[:,lat_counter,:],obsPerDay,\
        er_period[0],er_period[1],\
        er_wavenumber[0],er_wavenumber[1],\
        er_depth[0],er_depth[1],"er")
    print('#####################################################')
    print(np.min(er[:,lat_counter,:]),np.max(er[:,lat_counter,:]))

# # Reset FiltData
# filtData=np.zeros([filtData.time.size,len(lat),len(lon)],dtype='f')
# for x in range(0,filtData.shape[0]):
#     for y in range(0,filtData.shape[1]):
#         for z in range(0,filtData.shape[2]):
#             filtData[x,y,z]=backup[x,y,z]


##############################
Filtering (ER)

###########################################################################################
95.725464 

0 <xarray.Variable ()>
array(-15., dtype=float32)
Attributes:
    units:          degrees_north
    actual_range:   [ 90. -90.]
    long_name:      Latitude
    standard_name:  latitude
    axis:           Y
max :  22.734087 

95.725464 

100.317505 

#####################################################
<xarray.Variable ()>
array(-48.730106, dtype=float32) <xarray.Variable ()>
array(49.414886, dtype=float32)
###########################################################################################
97.36752 

1 <xarray.Variable ()>
array(-12.5, dtype=float32)
Attributes:
    units:          degrees_north
    actual_range:   [ 90. -90.]
    long_name:      Latitude
    standard_name:  latitude
    axis:           Y
max :  20.714346 

97.36752 

97.40141 

#####################################################
<xarray.Variable ()>
array(-

In [7]:
# #################################################
# # Write Out Data
# #################################################
# outPath = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/test.nc'
# outFile = nc.Dataset(outPath, mode='w')

# # Write Out Dimensions
# out_time = outFile.createDimension('time', infile['olr_anom'].time.size)
# out_lat = outFile.createDimension('lat', len(lat))
# out_lon = outFile.createDimension('lon', len(lon))


# print("\n##############################\nWriting Out Data\n")
# #ER
# out_er = outFile.createVariable('er', np.float32,('time','lat','lon'))
# out_er[:,:,:]=er[0:infile['olr_anom'].time.size,:,:]
# out_er.depth=er_depth
# out_er.period=er_period
# out_er.wavenumber=er_wavenumber
# out_er.filter=er_filter
# out_er.units=er_units
# out_er.long_name=er_longname

In [8]:
er
er = er.sel(time = '2009')


AttributeError: 'Variable' object has no attribute 'sel'

In [10]:
er_test = infile['er_wk']*0
er_test[:,:,:] = er[:,:,:]
er_test = er_test.sel(time = '2009')
er_test.to_netcdf('/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/ER_SECOND_3.nc')